In [26]:
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import os

credentials_file = r'C:\Users\XN6473\OneDrive - ENGIE\Documentos\ENGIE\analytics\favorable-bolt-205119-067ff6bc7b26.json'
scope = ['https://www.googleapis.com/auth/analytics.readonly']

view_id = '244486947'

credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_file, scope)
analytics = build('analyticsreporting', 'v4', credentials=credentials)

start_date = '2021-04-01'
end_date = '2023-06-30'

request = {
    'viewId': view_id,
    'dimensions': [
        {'name': 'ga:userType'},
        {'name': 'ga:sessionCount'},
        {'name': 'ga:date'},
        {'name': 'ga:city'},
        {'name': 'ga:region'},
        {'name': 'ga:country'},
        {'name': 'ga:referralPath'},
        {'name': 'ga:medium'},
        {'name': 'ga:operatingSystem'}
    ],
    'metrics': [
        {'expression': 'ga:visits'},
        {'expression': 'ga:newUsers'},
        {'expression': 'ga:users'},
        {'expression': 'ga:sessions'},
        {'expression': 'ga:avgSessionDuration'},
        {'expression': 'ga:bounces'}
    ],
    'pageSize': 100000
}

def get_report_data(request):
    dimensions_list = []
    metrics_list = []

    response = analytics.reports().batchGet(body={'reportRequests': [request]}).execute()
    data = response['reports'][0]['data']
    
    # Verifica se existem linhas de dados na resposta
    if 'rows' in data:
        rows = data['rows']
        for row in rows:
            dimensions = row['dimensions']
            metrics = row['metrics'][0]['values']
            dimensions_list.append(dimensions)
            metrics_list.append(metrics)
    
    next_page_token = data.get('nextPageToken')
    if next_page_token:
        request['pageToken'] = next_page_token
        dimensions_list_extend, metrics_list_extend = get_report_data(request)
        dimensions_list.extend(dimensions_list_extend)
        metrics_list.extend(metrics_list_extend)
    
    return dimensions_list, metrics_list

dataframes = []

current_date = start_date
while current_date <= end_date:
    year, month, _ = current_date.split('-')

    print(f'Processando dados para o período {month}-{year} a {int(month)+1}-{year}...')

    start_date_month = f'{year}-{month}-01'
    end_date_month = pd.to_datetime(start_date_month) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
    end_date_month = end_date_month.strftime('%Y-%m-%d')
    request['dateRanges'] = [{'startDate': start_date_month, 'endDate': end_date_month}]

    dimensions_list, metrics_list = get_report_data(request)

    df = pd.DataFrame(dimensions_list, columns=['userType', 'sessionCount', 'date', 'city', 'region', 'country', 'referralPath', 'medium', 'operatingSystem'])
    df_metrics = pd.DataFrame(metrics_list, columns=['visits', 'newUsers', 'users', 'sessions', 'avgSessionDuration', 'bounces'])
    df = pd.concat([df, df_metrics], axis=1)

    csv_filename = f'dados_analytics_{month}_{year}_to_{int(month)+1}_{year}.csv'
    
    i = 1
    while os.path.exists(csv_filename):
        csv_filename = f'dados_analytics_{month}_{year}_to_{int(month)+1}_{year}_{i}.csv'
        i += 1
    
    df.to_csv(csv_filename, index=False)
    print(f'Dados exportados para {csv_filename}')

    dataframes.append(df)

    current_date = pd.to_datetime(current_date) + pd.DateOffset(months=2)
    current_date = current_date.strftime('%Y-%m-%d')

combined_df = pd.concat(dataframes, ignore_index=True)

combined_df.drop_duplicates(inplace=True)

combined_csv_filename = 'energy_placer.csv'

df = combined_df
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.strftime('%d/%m/%Y')
df['avgSessionDuration'] = df['avgSessionDuration'].astype(float).round(3)
df['newUsers'] = df['newUsers'].astype(int)
df['users'] = df['users'].astype(int)
df['sessions'] = df['sessions'].astype(int)


Processando dados para o período 04-2021 a 5-2021...
Dados exportados para dados_analytics_04_2021_to_5_2021.csv
Processando dados para o período 06-2021 a 7-2021...
Dados exportados para dados_analytics_06_2021_to_7_2021.csv
Processando dados para o período 08-2021 a 9-2021...
Dados exportados para dados_analytics_08_2021_to_9_2021.csv
Processando dados para o período 10-2021 a 11-2021...
Dados exportados para dados_analytics_10_2021_to_11_2021.csv
Processando dados para o período 12-2021 a 13-2021...
Dados exportados para dados_analytics_12_2021_to_13_2021.csv
Processando dados para o período 02-2022 a 3-2022...
Dados exportados para dados_analytics_02_2022_to_3_2022.csv
Processando dados para o período 04-2022 a 5-2022...
Dados exportados para dados_analytics_04_2022_to_5_2022.csv
Processando dados para o período 06-2022 a 7-2022...
Dados exportados para dados_analytics_06_2022_to_7_2022.csv
Processando dados para o período 08-2022 a 9-2022...
Dados exportados para dados_analytics_0

In [27]:
df.to_csv('gralha', index=False)